# Modèle final (LSTM Bidirectionnal)

### Sommaire :
* Importation des packages
* Importation des données
* PRÉTRAITEMENT : VECTORISATION DU TEXTE
* CRÉATION DU DATASET TENSORFLOW
* CONSTRUCTION ET ENTRAÎNEMENT DU MODÈLE
* PRÉDICTIONS ET ÉVALUATIONS INITIALES
* CALCUL DES MÉTRIQUES AVEC KERAS
* CALCUL DES MÉTRIQUES AVEC SKLEARN
* SAUVEGARDE ET CHARGEMENT DU MODÈLE
* FONCTION DE SCORING D'UN COMMENTAIRE
* ÉVALUATION COMPLÉMENTAIRE AVEC ACCURACY_SCORE


# Importation des packages

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# Import des modules Keras et sklearn
from tensorflow.keras.layers import TextVectorization, LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras import Sequential
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [ ]:
# Importez le module de lecteur de Google.colab pour interagir avec Google Drive
from google.colab import drive

# Montez Google Drive vers le répertoire '/Content/Drive'
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Importation des données d'entraînement à partir de Google Drive
data = pd.read_csv('/Users/romain/Downloads/Classification/data_classification_commentaires_toxiques/train.csv')

# Affichage rapide du dataframe pour vérifier les données importées
print(data)

# Affichage d'un exemple de commentaire toxique pour la classe 'identity_hate'
print(data.loc[data.identity_hate == 1].iloc[0].comment_text)

# 2. PRÉTRAITEMENT : VECTORISATION DU TEXTE

In [ ]:
# Sélection des commentaires et des labels
x = data.comment_text  # Texte des commentaires
y = data[data.columns[2:]].values  # Labels des différentes classes de toxicité

# Définition du nombre maximum de caractéristiques (mots) à prendre en compte
max_words = 200000

# Initialisation du TextVectorization pour transformer le texte en séquences d'entiers
vectorizer = TextVectorization(max_tokens=max_words,
                               output_sequence_length=2000,
                               output_mode='int')

# Adapter le vectoriseur sur les données textuelles pour apprendre le vocabulaire
vectorizer.adapt(x.values)

# Affichage de la taille du vocabulaire appris
print("Taille du vocabulaire :", len(vectorizer.get_vocabulary()))

# Exemple de vectorisation d'un texte
print("Exemple de vectorisation :", vectorizer('Hello world, my name is Dualingo'))

# Conversion de tous les commentaires en séquences vectorisées
vectorized_text = vectorizer(x.values)

# 3. CRÉATION DU DATASET TENSORFLOW

In [ ]:
# Création d'un dataset TensorFlow à partir des données vectorisées et des labels
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
# Mise en cache du dataset pour améliorer les performances
dataset = dataset.cache()
# Mélange des données pour garantir que les batches sont aléatoires
dataset = dataset.shuffle(160000)
# Division des données en batches de taille 16
dataset = dataset.batch(16)
# Préchargement des données pour améliorer les performances d'entraînement
dataset = dataset.prefetch(8)

# Affichage d'un batch pour vérification
batch_x, batch_y = dataset.as_numpy_iterator().next()
print("Shape d'un batch (x):", batch_x.shape)
print("Shape d'un batch (y):", batch_y.shape)

# Division du dataset en ensembles d'entraînement, validation et test
total_batches = len(dataset)
train = dataset.take(int(total_batches * 0.7))  # 70% pour l'entraînement
val = dataset.skip(int(total_batches * 0.7)).take(int(total_batches * 0.2))  # 20% pour la validation
test = dataset.skip(int(total_batches * 0.9)).take(int(total_batches * 0.1))  # 10% pour le test

# 4. CONSTRUCTION ET ENTRAÎNEMENT DU MODÈLE

In [ ]:
# Création d'un modèle séquentiel
model = Sequential()
# Ajout d'une couche d'embedding pour convertir les indices de mots en vecteurs denses
model.add(Embedding(max_words + 1, 32))
# Ajout d'une couche LSTM bidirectionnelle pour capturer les dépendances dans les deux directions
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Ajout de couches denses avec activation ReLU pour l'apprentissage des caractéristiques complexes
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Ajout de la couche de sortie avec activation sigmoïde pour les prédictions binaires sur 6 classes
model.add(Dense(6, activation='sigmoid'))

# Affichage du résumé du modèle pour vérifier l'architecture
model.summary()

# Compilation du modèle avec une fonction de perte binaire et l'optimiseur Adam
model.compile(loss='BinaryCrossentropy', optimizer='Adam', metrics=['accuracy'])

# Entraînement du modèle sur les données d'entraînement avec validation sur les données de validation
history = model.fit(train, epochs=20, validation_data=val)

# 5. PRÉDICTIONS ET ÉVALUATIONS INITIALES

In [ ]:
# Prédiction sur un exemple de texte
input_text = vectorizer('You are so dumbt! if i see you, you are dead')
print("Prédiction (exemple 1) :", model.predict(np.array([input_text])))
print("Prédiction (exemple 2) :", model.predict(np.expand_dims(input_text, 0)))

# Evaluation sur un batch du test set
batch_x, batch_y = test.as_numpy_iterator().next()
print("Prédictions sur batch test :", (model.predict(batch_x) > 0.5).astype(int))
print("Labels réels :", batch_y)

# 6. CALCUL DES MÉTRIQUES AVEC KERAS

In [ ]:
# Initialisation des métriques de précision et de rappel
precision = Precision()
recall = Recall()

# Boucle sur chaque batch du jeu de données de test
for batch in test.as_numpy_iterator():
    batch_x, batch_y = batch
    # Prédiction des labels pour le batch courant
    yhat = model.predict(batch_x)
    # Aplatir les tableaux pour le calcul des métriques
    precision.update_state(batch_y.flatten(), yhat.flatten())
    recall.update_state(batch_y.flatten(), yhat.flatten())

# Calcul des résultats finaux pour la précision et le rappel
precision = precision.result().numpy()
recall = recall.result().numpy()
# Calcul du F1-score à partir de la précision et du rappel
f1score = (2 * precision * recall) / (precision + recall)

# Affichage des résultats
print(f"Precision: {precision}\nRecall: {recall}\nF1-score: {f1score}")

# 7. CALCUL DES MÉTRIQUES AVEC SKLEARN

In [ ]:
# Initialisation des listes pour stocker les vraies étiquettes et les prédictions
y_true = []
y_pred = []

# Boucle sur chaque batch du jeu de données de test
for batch in test.as_numpy_iterator():
    batch_x, batch_y = batch
    # Prédiction des labels pour le batch courant
    yhat = model.predict(batch_x)
    # Ajout des vraies étiquettes et des prédictions aux listes
    y_true.append(batch_y)
    y_pred.append(yhat)

# Conversion des listes en tableaux numpy
y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

# Seuil de 0.5 pour binariser les prédictions
y_pred_bin = (y_pred > 0.5).astype(int)

# Calcul des métriques par label
precision_per_label = precision_score(y_true, y_pred_bin, average=None)
recall_per_label = recall_score(y_true, y_pred_bin, average=None)
f1_per_label = f1_score(y_true, y_pred_bin, average=None)

# Affichage des résultats par label
print(f"Precision par label: {precision_per_label}")
print(f"Recall par label: {recall_per_label}")
print(f"F1-score par label: {f1_per_label}")

# Calcul des F1-scores globaux
f1_macro = f1_score(y_true, y_pred_bin, average="macro")
f1_micro = f1_score(y_true, y_pred_bin, average="micro")

# Affichage des F1-scores globaux
print(f"F1-score macro (moyenne): {f1_macro}")
print(f"F1-score micro (global): {f1_micro}")

# 8. SAUVEGARDE ET CHARGEMENT DU MODÈLE

In [ ]:
# sauvegarde du modele
model.save('Final_LSTM.h5')
# chargement du modele
model = tf.keras.models.load_model('Final_LSTM.h5')
print("Modèle chargé :", model)

# 9. FONCTION DE SCORING D'UN COMMENTAIRE

In [ ]:
def score_comment(comment):
    # Vectorisation du commentaire pour le transformer en séquence d'entiers
    vectorized_comment = vectorizer([comment])
    
    # Prédiction des probabilités de toxicité pour chaque classe
    results = model.predict(vectorized_comment)
    
    # Initialisation d'une chaîne de caractères pour stocker les résultats
    text = ""
    
    # Boucle sur chaque classe de toxicité pour formater les résultats
    for idx, col in enumerate(data.columns[2:]):
        # Ajout du nom de la classe et du résultat de la prédiction (True si > 0.5, sinon False)
        text += f'{col}: {results[0][idx] > 0.5}\n'
    
    # Retourne le texte formaté avec les résultats
    return text

# Exemple d'utilisation de la fonction avec un commentaire toxique
print(score_comment("FUCK YOU"))

# 10. ÉVALUATION COMPLÉMENTAIRE AVEC ACCURACY_SCORE

In [ ]:
# bout de code pour pouvoir évaluer des phrases seuls, suffit de changer txt
txt = "I hate you"
vec_txt = vectorizer(txt)
print("Prédiction pour la phrase mise est :", model.predict(np.expand_dims(vec_txt, 0)))
print("Labels :", data.columns[2:])

In [ ]:
# Préparation des données de test pour évaluer l'accuracy
x_test = np.expand_dims(vec_txt.numpy(), 0)
y_test = [[1, 0, 0, 0, 0, 0]]
for batch in test.as_numpy_iterator():
    batch_x, batch_y = batch
    x_test = np.concatenate((x_test, batch_x))
    y_test = np.concatenate((y_test, batch_y))
yhat = model.predict(x_test)
yhat = (yhat > 0.5).astype(int)
print("Accuracy :", accuracy_score(np.array(y_test).flatten(), yhat.flatten()))